In [44]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, text, select, insert, update, delete

# Function to read database credentials from a file
def read_db_creds(file_path):
    creds = {}
    with open(file_path, 'r') as f:
        for line in f:
            key, value = line.strip().split('=')
            creds[key] = value
    return creds

# Read credentials from dbcreds.txt

creds = read_db_creds('dbcreds.txt')


# Define the connection string
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
HOST = creds['host']
USER = creds['user']
PASSWORD = creds['password']
DATABASE = 'univ'
PORT = int(creds['port'])

# Create the connection string
connection_string = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"

# Create an engine that connects to the database
engine = create_engine(connection_string)


In [6]:
# nested subqueries
# subquery in a where clause
query = '''
select distinct course_id
from section
where semester = 'Fall' and year = 2017 and
course_id in (select course_id
from section
where semester = 'Spring' and year = 2018);
'''

In [7]:
df = pd.read_sql(query, engine)
df

,course_id
0,CS-101


In [8]:
# subquery in a where clause
query = '''
select distinct course_id
from section
where semester = 'Fall' and year = 2017 and
course_id not in (select course_id
from section
where semester = 'Spring' and year = 2018);
'''

In [9]:
df = pd.read_sql(query, engine)
df

,course_id
0,CS-347
1,PHY-101


In [19]:
query = 'select * from teaches;'
df = pd.read_sql(query, engine)
df.head()

,id,course_id,sec_id,semester,year
0,10101,CS-101,1,Fall,2017.0
1,10101,CS-315,1,Spring,2018.0
2,10101,CS-347,1,Fall,2017.0
3,12121,FIN-201,1,Spring,2018.0
4,15151,MU-199,1,Spring,2018.0


In [18]:
query = 'select * from takes;'
df = pd.read_sql(query, engine)
df.head()

,id,course_id,sec_id,semester,year,grade
0,00128,CS-101,1,Fall,2017.0,A
1,00128,CS-347,1,Fall,2017.0,A-
2,12345,CS-101,1,Fall,2017.0,C
3,12345,CS-190,2,Spring,2017.0,A
4,12345,CS-315,1,Spring,2018.0,A


In [20]:
# set membership
query = '''
select count (distinct id)
from takes
where (course_id, sec_id, semester, year) in
            (select course_id, sec_id, semester, year
            from teaches
            where teaches.id = '10101');
'''

df = pd.read_sql(query, engine)
df

,count
0,6


In [22]:
# some clause
query = '''
select distinct T.name
from instructor T, instructor S
where T.salary > S.salary and S.dept_name = 'Biology';
'''

df = pd.read_sql(query, engine)
df

,name
0,Einstein
1,Katz
2,Singh
3,Kim
4,Brandt
5,Wu
6,Gold


In [23]:
# some clause
query = '''
select name
from instructor
where salary > some(select salary
                    from instructor
                    where dept_name = 'Biology');
'''

df = pd.read_sql(query, engine)
df

,name
0,Wu
1,Einstein
2,Gold
3,Katz
4,Singh
5,Brandt
6,Kim


In [24]:
# all clause
query = '''
select name
from instructor
where salary > all(select salary
                    from instructor
                    where dept_name = 'Biology');
'''

df = pd.read_sql(query, engine)
df

,name
0,Wu
1,Einstein
2,Gold
3,Katz
4,Singh
5,Brandt
6,Kim


In [25]:
# exists
# subquery in a where clause
query = '''
select course_id
from section as S
where semester = 'Fall' and year = 2017 and
exists (select *
        from section T
        where semester = 'Spring' and year = 2018
        and S.course_id = T.course_id);
'''

df = pd.read_sql(query, engine)
df

,course_id
0,CS-101


In [26]:
# not exists
query = '''
select distinct S.id, S.name
from student S
where not exists ((select course_id
                  from course
                  where dept_name = 'Biology')
                  except
                  (select T.course_id
                    from takes T
                    where S.id = T.id));
'''

df = pd.read_sql(query, engine)
df

,id,name


In [30]:
# unique 
# unique not supported in postgresql
# query = '''
# select T.course_id
# from course T
# where unique (select R.course_id
#                 from section R
#                 where T.course_id = R.course_id
#                 and R.year = 2018);
# '''
query = '''
SELECT T.course_id
FROM course T
WHERE EXISTS (
	SELECT 1
	FROM section R
	WHERE T.course_id = R.course_id
	AND R.year = 2018
	GROUP BY R.course_id
	HAVING COUNT(DISTINCT R.course_id) = 1
);
'''
df = pd.read_sql(query, engine)
df

,course_id
0,BIO-301
1,CS-101
2,CS-315
3,CS-319
4,FIN-201
5,HIS-351
6,MU-199


In [32]:
# subquery in from clause
query = '''
select dept_name, avg_salary
from (select dept_name, avg(salary) as avg_salary
from instructor
group by dept_name)
where avg_salary > 42000;
'''

df = pd.read_sql(query, engine)
df

,dept_name,avg_salary
0,Finance,85000.000000
1,History,61000.000000
2,Physics,91000.000000
3,Comp. Sci.,77333.333333
4,Biology,72000.000000
5,Elec. Eng.,80000.000000


In [33]:
# subquery in from clause
query = '''
select dept_name, avg_salary
from (select dept_name, avg(salary)
        from instructor
        group by dept_name) as dept_avg (dept_name, avg_salary)
where avg_salary > 42000;
'''

df = pd.read_sql(query, engine)
df

,dept_name,avg_salary
0,Finance,85000.000000
1,History,61000.000000
2,Physics,91000.000000
3,Comp. Sci.,77333.333333
4,Biology,72000.000000
5,Elec. Eng.,80000.000000


In [35]:
# with clause
query = '''
with max_budget(value) as
    (select max(budget)
    from department)
select dept_name
from department, max_budget
where department.budget = max_budget.value;
'''
df = pd.read_sql(query, engine)
df

,dept_name
0,Finance


In [41]:
# with clause
query = '''
with dept_total(dept_name, value) as
        (select dept_name, sum(salary)
        from instructor
        group by dept_name),
     dept_total_avg(value) as
        (select avg(value)
        from dept_total)
select dept_name
from dept_total, dept_total_avg
where dept_total.value > dept_total_avg.value;
'''

df = pd.read_sql(query, engine)
df

,dept_name
0,Finance
1,Physics
2,Comp. Sci.


In [51]:
# with clause
query = '''
with dept_total(dept_name, value) as
        (select dept_name, sum(salary)
        from instructor
        group by dept_name),
     dept_total_avg(value) as
        (select avg(value)
        from dept_total)
select count(*)
from dept_total, dept_total_avg
where dept_total.value > dept_total_avg.value;
'''

df = pd.read_sql(query, engine)
df

,count
0,3


In [43]:
#  scalar subquery
query = '''
select dept_name,
    (select count(*)
    from instructor
    where department.dept_name = instructor.dept_name)
    as num_instructors
    from department;
'''

df = pd.read_sql(query, engine)
df

,dept_name,num_instructors
0,Biology,1
1,Comp. Sci.,3
2,Physics,2
3,Music,1
4,History,2
5,Finance,2
6,Elec. Eng.,1


In [3]:
# get tables in the database
query = '''
select *
from information_schema.tables
where table_schema = 'public';
'''

df = pd.read_sql(query, engine)
df.iloc[:, 2]

0     department
1         prereq
2          takes
3        teaches
4         course
5        section
6      classroom
7      time_slot
8     instructor
9        student
10       advisor
Name: table_name, dtype: object

In [5]:
query = 'select * from course;'
df = pd.read_sql(query, engine)
df

,course_id,title,dept_name,credits
0,BIO-101,Intro. to Biology,Biology,4.0
1,BIO-301,Genetics,Biology,4.0
2,BIO-399,Computational Biology,Biology,3.0
3,CS-101,Intro. to Computer Science,Comp. Sci.,4.0
4,CS-190,Game Design,Comp. Sci.,4.0
5,CS-315,Robotics,Comp. Sci.,3.0
6,CS-319,Image Processing,Comp. Sci.,3.0
7,CS-347,Database System Concepts,Comp. Sci.,3.0
8,EE-181,Intro. to Digital Systems,Elec. Eng.,3.0
9,FIN-201,Investment Banking,Finance,3.0


In [19]:
sql_command = text('''
CREATE TABLE course_2 AS
SELECT * FROM course
WHERE course_id IN ('BIO-301', 'CS-190', 'CS-315');
''')

# Obtain a connection from the engine
with engine.connect() as connection:
	# Execute the SQL command
	connection.execute(sql_command)
	connection.commit()

In [21]:
sql_command = text('''
CREATE TABLE prereq_2 AS
SELECT * FROM prereq
WHERE course_id IN ('BIO-301', 'CS-190', 'CS-347');
''')

# Obtain a connection from the engine
with engine.connect() as connection:
    # Execute the SQL command
    connection.execute(sql_command)
    connection.commit()

In [22]:
# get tables in the database
query = '''
select *
from information_schema.tables
where table_schema = 'public';
'''

df = pd.read_sql(query, engine)
df.iloc[:, 2]

0     department
1         prereq
2          takes
3        teaches
4         course
5       course_2
6       prereq_2
7        section
8      classroom
9      time_slot
10    instructor
11       student
12       advisor
Name: table_name, dtype: object

In [23]:
query = 'select * from course_2;'
df = pd.read_sql(query, engine)
df

,course_id,title,dept_name,credits
0,BIO-301,Genetics,Biology,4.0
1,CS-190,Game Design,Comp. Sci.,4.0
2,CS-315,Robotics,Comp. Sci.,3.0


In [24]:
query = 'select * from prereq_2;'
df = pd.read_sql(query, engine)
df

,course_id,prereq_id
0,BIO-301,BIO-101
1,CS-190,CS-101
2,CS-347,CS-101


In [31]:
# inner join
query = '''
SELECT *
FROM course_2
INNER JOIN prereq_2 ON prereq_2.course_id = course_2.course_id;
'''
df = pd.read_sql(query, engine)
df

,course_id,title,dept_name,credits,course_id,prereq_id
0,BIO-301,Genetics,Biology,4.0,BIO-301,BIO-101
1,CS-190,Game Design,Comp. Sci.,4.0,CS-190,CS-101


In [32]:
# natural join
query = '''
SELECT *
FROM course_2
NATURAL JOIN prereq_2;
'''

df = pd.read_sql(query, engine)
df

,course_id,title,dept_name,credits,prereq_id
0,BIO-301,Genetics,Biology,4.0,BIO-101
1,CS-190,Game Design,Comp. Sci.,4.0,CS-101


In [33]:
# OUTER JOIN
# LEFT OUTER JOIN
query = '''
SELECT *
FROM course_2
NATURAL LEFT OUTER JOIN prereq_2;
'''
df = pd.read_sql(query, engine)
df

,course_id,title,dept_name,credits,prereq_id
0,BIO-301,Genetics,Biology,4.0,BIO-101
1,CS-190,Game Design,Comp. Sci.,4.0,CS-101
2,CS-315,Robotics,Comp. Sci.,3.0,None


In [34]:
# RIGHT OUTER JOIN
query = '''
SELECT *
FROM course_2
NATURAL RIGHT OUTER JOIN prereq_2;
'''
df = pd.read_sql(query, engine)
df

,course_id,title,dept_name,credits,prereq_id
0,BIO-301,Genetics,Biology,4.0,BIO-101
1,CS-190,Game Design,Comp. Sci.,4.0,CS-101
2,CS-347,None,None,NaN,CS-101


In [35]:
# full outer join
query = '''
SELECT *
FROM course_2
NATURAL FULL OUTER JOIN prereq_2;
'''
df = pd.read_sql(query, engine)
df

,course_id,title,dept_name,credits,prereq_id
0,BIO-301,Genetics,Biology,4.0,BIO-101
1,CS-190,Game Design,Comp. Sci.,4.0,CS-101
2,CS-347,None,None,NaN,CS-101
3,CS-315,Robotics,Comp. Sci.,3.0,None


In [36]:
query = '''
SELECT *
FROM course_2
INNER JOIN prereq_2 ON course_2.course_id = prereq_2.course_id;
'''
df = pd.read_sql(query, engine)
df

,course_id,title,dept_name,credits,course_id,prereq_id
0,BIO-301,Genetics,Biology,4.0,BIO-301,BIO-101
1,CS-190,Game Design,Comp. Sci.,4.0,CS-190,CS-101


In [37]:
query = '''
SELECT *
FROM course_2
LEFT OUTER JOIN prereq_2 ON course_2.course_id = prereq_2.course_id;
'''
df = pd.read_sql(query, engine)
df

,course_id,title,dept_name,credits,course_id,prereq_id
0,BIO-301,Genetics,Biology,4.0,BIO-301,BIO-101
1,CS-190,Game Design,Comp. Sci.,4.0,CS-190,CS-101
2,CS-315,Robotics,Comp. Sci.,3.0,None,None


In [38]:
query = '''
SELECT *
FROM course_2
NATURAL RIGHT OUTER JOIN prereq_2;
'''
df = pd.read_sql(query, engine)
df

,course_id,title,dept_name,credits,prereq_id
0,BIO-301,Genetics,Biology,4.0,BIO-101
1,CS-190,Game Design,Comp. Sci.,4.0,CS-101
2,CS-347,None,None,NaN,CS-101


In [39]:
query = '''
SELECT *
FROM course_2
FULL OUTER JOIN prereq_2 using (course_id);
'''
df = pd.read_sql(query, engine)
df

,course_id,title,dept_name,credits,prereq_id
0,BIO-301,Genetics,Biology,4.0,BIO-101
1,CS-190,Game Design,Comp. Sci.,4.0,CS-101
2,CS-347,None,None,NaN,CS-101
3,CS-315,Robotics,Comp. Sci.,3.0,None


In [46]:
view = text('''
create view faculty as
    select ID, name, dept_name
    from instructor;
''')

with engine.connect() as connection:
    connection.execute(view)
    connection.commit()

In [48]:
# Now, query the newly created view
query = 'SELECT * FROM faculty;'
df = pd.read_sql(query, engine)
df

,id,name,dept_name
0,10101,Srinivasan,Comp. Sci.
1,12121,Wu,Finance
2,15151,Mozart,Music
3,22222,Einstein,Physics
4,32343,El Said,History
5,33456,Gold,Physics
6,45565,Katz,Comp. Sci.
7,58583,Califieri,History
8,76543,Singh,Finance
9,76766,Crick,Biology


In [49]:
query = '''
select name
    from faculty
    where dept_name = 'Biology';
'''
df = pd.read_sql(query, engine)
df

,name
0,Crick


In [50]:
view2 = text('''
create view departments_total_salary(dept_name, total_salary) as
    select dept_name, sum(salary)
    from instructor
    group by dept_name;
''')

with engine.connect() as connection:
    connection.execute(view2)
    connection.commit()

In [51]:
query = 'SELECT * FROM departments_total_salary;'
df = pd.read_sql(query, engine)
df

,dept_name,total_salary
0,Finance,170000.0
1,History,122000.0
2,Physics,182000.0
3,Music,40000.0
4,Comp. Sci.,232000.0
5,Biology,72000.0
6,Elec. Eng.,80000.0
